In [51]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "id": "ea54fca2-3a3e-4b3b-91e7-a7bf971e0443",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "user": {
      "name": "Philliplakaschus@gmail.com",
      "type": "user"
    }
  }
]


In [52]:
subscription_id = !az account show --query id --output tsv
subscription_id = subscription_id[0]

In [79]:
# Get the resource group name and workspace name for the workspace from the config.conf file using pyhocon
from pyhocon import ConfigFactory
config = ConfigFactory.parse_file("config.conf")
resource_group = config.get_string("RESOURCE_GROUP")
workspace_name = config.get_string("WORKSPACE_NAME")

In [81]:

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

# NOTE: It's very import to always set the resource_group and workspace_name when creating the MLClient
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)
print(ml_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001DE4AF76710>,
         subscription_id=ea54fca2-3a3e-4b3b-91e7-a7bf971e0443,
         resource_group_name=rg-nano-gpt,
         workspace_name=ml-nano-gpt)


## Training

In [84]:
from azure.ai.ml.entities import Environment

custom_env = Environment(
    name='nano_gpt_env',
    description="Custom environment for nano gpt training",
    conda_file="../environment.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.8-cudnn8-ubuntu22.04"
)
ml_client.environments.create_or_update(custom_env)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'nano_gpt_env', 'description': 'Custom environment for nano gpt training', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/ea54fca2-3a3e-4b3b-91e7-a7bf971e0443/resourceGroups/rg-nano-gpt/providers/Microsoft.MachineLearningServices/workspaces/ml-nano-gpt/environments/nano_gpt_env/versions/1', 'Resource__source_path': None, 'base_path': 'c:\\Users\\d92343\\Documents\\IT\\Projects\\Private\\nanoGPTExperiments\\azure_deployment', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001DE4AF3B2B0>, 'serialize': <msrest.serialization.Serializer object at 0x000001DE4AF38E50>, 'version': '1', 'latest_version': None, 'conda_file': {'dependencies': ['python=3.10', 'pip', {'pip': ['numpy', 'transformers', 'datasets', 'tiktoken', 'mlflow', 'tqdm', 'azure-ai-ml', 'torch', 'pyhocon']}]

In [88]:
from azure.ai.ml import command

cmd = "python train.py --batch_size=32 --compile=False --device=gpu"
# configure job
job = command(
    code="../src",
    command=cmd,
    environment=custom_env,
    compute="aml-cluster-nano-gpt",
    display_name="nano-gpt-training",
    experiment_name="nano-gpt-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (27.81 MBs): 100%|##########| 27806177/27806177 [00:07<00:00, 3901002.81it/s]




Monitor your job at https://ml.azure.com/runs/purple_farm_yg8jxpx2ws?wsid=/subscriptions/ea54fca2-3a3e-4b3b-91e7-a7bf971e0443/resourcegroups/rg-nano-gpt/workspaces/ml-nano-gpt&tid=1fdb6d07-480b-4c31-a913-e847d51e7446


In [89]:
returned_job

Experiment,Name,Type,Status,Details Page
nano-gpt-training,purple_farm_yg8jxpx2ws,command,Starting,Link to Azure Machine Learning studio
